In [ ]:
# client
import socket
import json
import base64
from cryptography.fernet import Fernet

# Create a UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)

# Load the encryption key
def load_key():
    with open("secret.key", "rb") as key_file:
        return key_file.read()

KEY = load_key()
cipher = Fernet(KEY)

# User authentication
username = input("Enter username: ")
password = input("Enter password: ")

auth_data = json.dumps({"type": "auth", "username": username, "password": password})
client_socket.sendto(auth_data.encode(), server_address)  # Send authentication request

# Receive authentication response
response, _ = client_socket.recvfrom(4096)
response = json.loads(response.decode())
if response["status"] == "success":
    print("Authentication successful. You can now chat.")
    while True:
        message = input("You: ")  # Take user input for chat
        encrypted_msg = base64.urlsafe_b64encode(cipher.encrypt(message.encode())).decode()
        chat_data = json.dumps({"type": "message", "content": encrypted_msg})
        client_socket.sendto(chat_data.encode(), server_address)  # Send encrypted message to server
else:
    print("Authentication failed.")

client_socket.close()  # Close the socket after use

Authentication successful. You can now chat.
